<a href="https://colab.research.google.com/github/hansglick/book_errata/blob/main/hf_utilisation_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import transformers

In [ ]:
 * url : https://huggingface.co/course/fr/chapter2/5?fw=tf
 * video : https://www.youtube.com/watch?v=ROxrFOEbsQE

# Lots d'observations (batch)

Les modèles de Hugging Face ne prennent en entrée que des tensors. Ceux-ci doivent être batchés à savoir que la première dimension du batch doit être reservé pour spécifier le nombre d'observations.

```python
tokenized_inputs = tokenizer(sequence, return_tensors="tf")
print(tokenized_inputs["input_ids"])
# Une dimension a bien été rajouté lors de l'application du tokenizer afin de bien livrer un batch au modele
```

## Padding, en détails

Il est nécessaire de padder les observations, car les tensors sont des objects *rectangulaires*. Et on le répète, les inputs des modèles sont des tensors. Ainsi on padd, i.e. on rajoute autant de tokens que nécessaire (le plus souvent à droite) afin d'obtenir des phrases de même longueur que la phrase la plus longue du jeu de données. L'identifiant du token de padd est accessible ici `tokenizer.pad_token_id` Afin d'avertir le modèle que les poids liés aux tokens de padd ne sont pas à prendre en compte lors du calcul de la loss, on doit également fourinir un masque d'attention qui spécifie quels sont les tokens à prendre en compte et ceux à ne pas prendre en compte. Le masque d'attention doit lui aussi être de la forme d'un batch évidemment

```python
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(tf.constant(batched_ids), attention_mask=tf.constant(attention_mask))
print(outputs.logits)
```

```python
tf.Tensor(
[[ 1.5693681  -1.3894582 ]
 [ 0.5803021  -0.41252586]], shape=(2, 2), dtype=float32)
```

En utilisant l'argument padding = True, tout se fait en transparence. Je reste sur ma faim car il explique pas pourquoi on n'a pas besoin de padder jusqu'à la taille limite sur lequel le transformer a été trained

```python
tokenizer(sentences,padding=True))
```

## Taille limite des séquences

Les transformeurs ont des longueurs limite définies. Si on leur donne en input des longueurs qui excedent la taille limite, ils planteront. 2 options s'offrent à nous pour traiter les longues séquences :
 * les tronquer, de sorte à ce qu'elles ne dépassent jamais la taille limite
 * utiliser des transformers adequates comme le LongFormer ou le LED voir https://huggingface.co/docs/transformers/model_doc/longformer et https://huggingface.co/docs/transformers/model_doc/led